In [1]:
import numpy as np
import pandas as pd
import pickle
import networkx as nx 
import osmnx as ox 
import numpy as np 
import seaborn as sns 
import numpy as np
from scipy.spatial.distance import cdist
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point, LineString

In [4]:
#Load volume dataset
traffic_df=pd.read_excel("/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/TrafficVolume/data/weekly_data_xl.xlsx")
traffic_df.drop(columns='Unnamed: 0', inplace=True)

# Load graph
with open("/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/data/BaseGraph_eRoad.pickle", 'rb') as file:
    H = pickle.load(file)

#H = pd.read_pickle("/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/data/BaseGraph_eRoad.pickle")
traffic_df.head()

/var/folders/67/gxr6c90j7x9_c9pz8lgvd8n40000gn/T/ipykernel_20970/3135082987.py:7: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  H = pickle.load(file)


,id,name,lat,lon,volume,year,month,volume1,volume2
0,41379V1201935,MESNADALSARMEN,61.115044,10.456279,13023,2018,1,6512,6512
1,41379V1201935,MESNADALSARMEN,61.115044,10.456279,13998,2018,2,6999,6999
2,41379V1201935,MESNADALSARMEN,61.115044,10.456279,13878,2018,3,6939,6939
3,41379V1201935,MESNADALSARMEN,61.115044,10.456279,14288,2018,4,7144,7144
4,41379V1201935,MESNADALSARMEN,61.115044,10.456279,14669,2018,5,7334,7334


In [41]:
traffic_points = traffic_df[['id', 'lat', 'lon']].drop_duplicates("id")
print(len(traffic_points))
traffic_points.head()

641


,id,lat,lon
0,41379V1201935,61.115044,10.456279
12,43294V181263,59.629705,9.499576
24,76082V578594,63.542709,10.862757
36,21376V2607269,59.484949,11.745634
48,68771V2258479,59.164143,10.199460


In [ ]:
# Assuming your network is already defined as a variable H
# Create an empty dataframe to store node attributes
node_df = pd.DataFrame(columns=['node', "osmid", "latitude", "longitude", 'gradient', "isBridge", "isTunnel", "roadclass", "geometry"])

# Iterate over all nodes in the network
# Note: Generates a lot of output - just erase it after the code has run 
for node in H.nodes:
    # Get the gradient attribute of the node and add it to the dataframe
    osmid = H.nodes[node]["osmid"]
    lat = H.nodes[node]["x"]
    lon = H.nodes[node]["y"]
    isBridge = H.nodes[node]["isBridge"]
    isTunnel = H.nodes[node]["isTunnel"]
    roadclass = H.nodes[node]["roadclass"]
    gradient = H.nodes[node]['elevation']
    geo = H.nodes[node]["geometry"]

    node_df = node_df.append({"osmid": osmid, 'node': node, "latitude": lat, "longitude": lon, 'gradient': gradient, "isBridge": isBridge, "isTunnel": isTunnel, "roadclass": roadclass, "geometry": geo}, ignore_index=True)

In [42]:
node_df

,node,osmid,latitude,longitude,gradient,isBridge,isTunnel,roadclass,geometry
0,155191,155191,11.127466,60.786205,143.160,0,0,0,POINT (11.127465806112035 60.78620456025842)
1,155201,155201,11.127185,60.786429,142.700,0,0,0,POINT (11.127184981853482 60.78642854398455)
2,155273,155273,11.127535,60.786334,142.705,0,0,0,POINT (11.127534712389643 60.78633428345484)
3,155289,155289,11.127400,60.786416,142.543,0,0,0,POINT (11.127399824588059 60.786415782679235)
4,155971,155971,11.993829,60.189189,155.408,0,0,0,POINT (11.993829462815306 60.189188517538796)
...,...,...,...,...,...,...,...,...,...
6939,1854916,1854916,26.838489,69.886671,150.777,0,0,0,POINT (26.838488991005768 69.88667056375346)
6940,1854917,1854917,22.804535,69.938777,0.343,0,0,0,POINT (22.80453542616325 69.93877650993828)
6941,1854918,1854918,23.314782,68.781654,13.682,0,0,0,POINT (23.314781714276098 68.78165369157571)
6942,1854919,1854919,15.827419,67.811673,32.114,0,0,0,POINT (15.827418912265161 67.81167344929793)


In [11]:
# Convert the edge data to a pandas DataFrame
edge_data = H.edges(data=True)
edge_df = pd.DataFrame(edge_data, columns=['source', 'target', 'data_dict'])

# Extract the data into separate columns
edge_df = pd.concat([edge_df.drop('data_dict', axis=1), edge_df['data_dict'].apply(pd.Series)], axis=1)

# Print the resulting DataFrame
edge_df.head()


,source,target,id,oneway,ref,name,funcroadclass,roadclass,isFerry,isBridge,isTunnel,speedlim,drivetime,length,length_weight,u,v,grade,grade_abs,geometry
0,155191,155273,138086,False,Kjonerudvegen,Kjonerudvegen,0,0,0,0,0,60,0.017585,15.291208,15.291208,155191,155273,-0.03,0.03,NaN
1,155191,351348,"[363590, 2168667, 362740, 138014]",False,E6,E6,0,0,0,0,0,110,"[0.19838599168327925, 0.08097851727109437, 0.0...",485.743882,"[129.09618695391856, 35.40442062748214, 4.9757...","[155192, 351173, 351174, 155191]","[155192, 351348, 351173, 351174]","[0.034, 0.013, -0.002, 0.018]","[0.034, 0.013, 0.002, 0.018]",LINESTRING (11.127465806112035 60.786204560258...
2,155191,155201,"[138150, 138118, 138126, 138134, 138142]",False,Kjonerudvegen,Kjonerudvegen,0,0,0,0,0,60,"[0.020742835002232715, 0.007996503314164363, 0...",54.859300,"[3.7483329629162823, 4.644146853710842, 6.9534...","[155201, 155337, 155317, 24154, 155327]","[155337, 155317, 155191, 24154, 155327]","[0.031, -0.018, 0.027, 0.005, 0.018]","[0.031, 0.018, 0.005, 0.027]",LINESTRING (11.127184981853482 60.786428543984...
3,155201,155289,138110,False,Kjonerudvegen,Kjonerudvegen,0,0,0,0,0,60,0.013559,11.790339,11.790339,155289,155201,0.013,0.013,NaN
4,155201,351618,"[2168673, 138022, 362732, 363598, 2168670]",False,E6,E6,0,0,0,0,0,110,"[0.025416070719352715, 0.16287845422361258, 0....",377.468769,"[65.06593976101533, 259.6613038347447, 4.91549...","[155200, 351618, 1854868, 351165, 351166]","[155200, 155201, 1854868, 351165, 351166]","[0.028, 0.056, 0.025, 0.053, 0.042]","[0.028, 0.056, 0.025, 0.053, 0.042]",LINESTRING (11.125266280578165 60.789677274131...


In [12]:
geo_trafficPoints = gpd.GeoDataFrame(traffic_points, geometry=gpd.points_from_xy(traffic_points['lon'], traffic_points['lat']))
geo_trafficPoints.head()

,id,lat,lon,geometry
0,41379V1201935,61.115044,10.456279,POINT (10.45628 61.11504)
12,43294V181263,59.629705,9.499576,POINT (9.49958 59.62971)
24,76082V578594,63.542709,10.862757,POINT (10.86276 63.54271)
36,21376V2607269,59.484949,11.745634,POINT (11.74563 59.48495)
48,68771V2258479,59.164143,10.199460,POINT (10.19946 59.16414)


In [39]:
traffic_df

,id,name,lat,lon,volume,year,month,volume1,volume2
0,41379V1201935,MESNADALSARMEN,61.115044,10.456279,13023,2018,1,6512,6512
1,41379V1201935,MESNADALSARMEN,61.115044,10.456279,13998,2018,2,6999,6999
2,41379V1201935,MESNADALSARMEN,61.115044,10.456279,13878,2018,3,6939,6939
3,41379V1201935,MESNADALSARMEN,61.115044,10.456279,14288,2018,4,7144,7144
4,41379V1201935,MESNADALSARMEN,61.115044,10.456279,14669,2018,5,7334,7334
...,...,...,...,...,...,...,...,...,...
7268,22331V248904,Eksetstranda,62.159291,6.042887,9759,2018,8,4880,4880
7269,22331V248904,Eksetstranda,62.159291,6.042887,9677,2018,9,4838,4838
7270,22331V248904,Eksetstranda,62.159291,6.042887,9630,2018,10,4815,4815
7271,22331V248904,Eksetstranda,62.159291,6.042887,9645,2018,11,4822,4822


In [16]:
# Drop all rows with NaN values in the 'column_name' column
edge_df2 = edge_df.dropna(subset=['geometry'])
edge_df2.head()

,source,target,id,oneway,ref,name,funcroadclass,roadclass,isFerry,isBridge,isTunnel,speedlim,drivetime,length,length_weight,u,v,grade,grade_abs,geometry
1,155191,351348,"[363590, 2168667, 362740, 138014]",False,E6,E6,0,0,0,0,0,110,"[0.19838599168327925, 0.08097851727109437, 0.0...",485.743882,"[129.09618695391856, 35.40442062748214, 4.9757...","[155192, 351173, 351174, 155191]","[155192, 351348, 351173, 351174]","[0.034, 0.013, -0.002, 0.018]","[0.034, 0.013, 0.002, 0.018]",LINESTRING (11.127465806112035 60.786204560258...
2,155191,155201,"[138150, 138118, 138126, 138134, 138142]",False,Kjonerudvegen,Kjonerudvegen,0,0,0,0,0,60,"[0.020742835002232715, 0.007996503314164363, 0...",54.859300,"[3.7483329629162823, 4.644146853710842, 6.9534...","[155201, 155337, 155317, 24154, 155327]","[155337, 155317, 155191, 24154, 155327]","[0.031, -0.018, 0.027, 0.005, 0.018]","[0.031, 0.018, 0.005, 0.027]",LINESTRING (11.127184981853482 60.786428543984...
4,155201,351618,"[2168673, 138022, 362732, 363598, 2168670]",False,E6,E6,0,0,0,0,0,110,"[0.025416070719352715, 0.16287845422361258, 0....",377.468769,"[65.06593976101533, 259.6613038347447, 4.91549...","[155200, 351618, 1854868, 351165, 351166]","[155200, 155201, 1854868, 351165, 351166]","[0.028, 0.056, 0.025, 0.053, 0.042]","[0.028, 0.056, 0.025, 0.053, 0.042]",LINESTRING (11.125266280578165 60.789677274131...
5,155273,155289,"[138102, 138094]",False,Kjonerudvegen,Kjonerudvegen,0,0,0,0,0,60,"[0.007400319249563996, 0.006224889557635327]",11.848008,"[5.412947441422023, 6.435060217012172]","[155273, 155281]","[155281, 155289]","[-0.022, -0.007]","[0.022, 0.007]",LINESTRING (11.127534712389643 60.786334283454...
6,155273,191524,"[176720, 176745, 176733]",False,Kjonerudvegen,Kjonerudvegen,0,0,0,0,0,60,"[0.005229397871931829, 0.01727116575823345, 0....",19.941540,"[0.3758324090894773, 4.547302497332026, 15.018...","[155273, 191515, 191500]","[191515, 191500, 191524]","[-0.037, 0.0, -0.03]","[0.037, 0.0, 0.03]",LINESTRING (11.127534712389643 60.786334283454...


In [33]:
nearest_point_index = np.argmin(nearest_points.values)

In [ ]:
from shapely.geometry import Point, LineString, MultiPoint
import numpy as np

# Assuming your LineStrings are stored in a GeoDataFrame called 'line_strings_df'
# and your geometry points are stored in a GeoDataFrame called 'points_df'

# Convert the GeoDataFrame of geometry points to a MultiPoint object
points = MultiPoint(geo_trafficPoints['geometry'].values)

# Compute the nearest points between each LineString and the MultiPoint
nearest_points = edge_df['geometry'].apply(lambda x: points.distance(x).min())

# Get the index of the nearest point for each LineString
nearest_point_index = np.argmin(nearest_points.values)

# Get the actual nearest point from the MultiPoint
nearest_point = points[nearest_point_index]

# Print the nearest point
print("Nearest point:", nearest_point)
